In [1]:
import torch
import json
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

from src import dataio
from src.fn_modeling import BertForFrameIdentification

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
MAX_LEN = 256
batch_size = 8
language = 'ko'
version = 1.1

framenet = 'kfn'
framenet_data = 'Korean FrameNet '+str(version)

# save your model to
model_dir = './models/'+framenet+'/'
result_dir = './result/'

print('### SETINGS')
print('\t# FrameNet:', framenet_data)
print('\t# model will be saved to', model_dir)
print('\t# result will be saved to', result_dir)

### SETINGS
	# FrameNet: Korean FrameNet 1.1
	# model will be saved to ./models/kfn/
	# result will be saved to ./result/


# Load Data

In [5]:
from koreanframenet import koreanframenet
if language == 'ko':
    kfn = koreanframenet.interface(version=version)
    trn, dev, tst = kfn.load_data()

### Korean FrameNet ###
	# contact: hahmyg@kaist, hahmyg@gmail.com #


### loading Korean FrameNet 1.1 data...
	# of instances in training data: 17838
	# of instances in dev data: 2548
	# of instances in test data: 5097


In [7]:
data_path = './koreanframenet/resource/info/'

with open(data_path+framenet+str(version)+'_lu2idx.json','r') as f:
    lu2idx = json.load(f)
with open(data_path+'fn1.7_frame2idx.json','r') as f:
    frame2idx = json.load(f)      
with open(data_path+framenet+str(version)+'_lufrmap.json','r') as f:
    lufrmap = json.load(f)
with open(data_path+framenet+str(version)+'_frargmap.json','r') as f:
    frargmap = json.load(f)
    
idx2frame = dict(zip(frame2idx.values(),frame2idx.keys()))
idx2lu = dict(zip(lu2idx.values(),lu2idx.keys()))
        
print('\nData Statistics...')
print('\t# of lu:', len(lu2idx))


Data Statistics...
	# of lu: 5489


# Load BERT tokenizer

In [8]:
# load pretrained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

# bert tokenizer
def bert_tokenizer(text):
    orig_tokens = text.split(' ')
    bert_tokens = []
    orig_to_tok_map = []
    bert_tokens.append("[CLS]")
    for orig_token in orig_tokens:
        orig_to_tok_map.append(len(bert_tokens))
        bert_tokens.extend(tokenizer.tokenize(orig_token))
    bert_tokens.append("[SEP]")
    
    return orig_tokens, bert_tokens, orig_to_tok_map

# Generate BERT input representation